<a href="https://colab.research.google.com/github/xyzkpsf/CS-W182-CV-Project/blob/main/adv_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import torchbearer
except:
    !pip install -q torchbearer
    import torchbearer

!pip install livelossplot

In [ ]:
try:
    from art.attacks.evasion import FastGradientMethod
    from art.estimators.classification import PyTorchClassifier
except:
    !pip install adversarial-robustness-toolbox
    from art.attacks.evasion import FastGradientMethod
    from art.estimators.classification import PyTorchClassifier

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
import pandas as pd
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torchvision.io import read_image
from skimage import io, transform
from PIL import Image
import matplotlib.pyplot as plt
from livelossplot import PlotLosses
import time
import copy
import os
import sys


In [ ]:
!unzip '/content/drive/MyDrive/Spring 2021/CS 182/tiny-imagenet-200.zip'

In [6]:
class CustomImageDataset(Dataset):
    def __init__(self, val_label_dir, img_dir, train_path, transform=None, target_transform=None):
        super(CustomImageDataset, self).__init__()
        self.val_label_file = pd.read_csv(val_label_dir, delimiter = "\t", names=["pics", "labels", "_1", "_2", "_3", "_4"])
        self.img_labels = self.val_label_file[["pics", 'labels']]
        self.img_dir = img_dir
        self.transform = transform
        self.classes, self.class_to_idx = self._find_classes(train_path)

    def __len__(self):
        return len(self.img_labels)

    def _find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
        """
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def pad(self, img):
        padding = np.ones((64, 64, 2))
        img = img.reshape((64, 64, 1))
        img = np.concatenate((img, padding), axis=2)
        return img.astype(np.float32)
      
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        img = Image.open(img_path)
        img = copy.deepcopy(np.asarray(img))
        # if it has less than 3 channels
        if img.shape != (64, 64, 3):
            img = self.pad(img)
        #print(img.shape)
        label = self.img_labels.iloc[idx, 1]
        label = self.class_to_idx[label]
        if self.transform:
            img = self.transform(img)
        #sample = {"image": img, "label": label}
        return img, label

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/tiny-imagenet-200'

val_label_dir = '/content/tiny-imagenet-200/val/val_annotations.txt'

image_datasets = {}

image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                          data_transforms['train'])

image_datasets['val'] = CustomImageDataset(val_label_dir, data_dir+'/val/images', 
                                           os.path.join(data_dir, 'train'),
                                           transform=data_transforms['val'])

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                             shuffle=True, num_workers=2)
                                            for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
model = torch.load("/content/drive/MyDrive/CV Project Model State Dict/Resnet18_WholeModel_val_54.pt")

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [42]:
classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer_ft,
    #input_shape=(3, 64, 64),
    input_shape=(64, 64, 3),
    nb_classes=200,
    device_type = device,
)

In [43]:
for epoch in range(1):
    for i,(inputs, labels) in enumerate(dataloaders['train']):
        inputs = inputs.numpy()
        labels = labels.numpy()
        classifier.fit(inputs, labels)
        if i % 10 == 0:
            print("EPOCH: {}, ITERATION: {}/{}".format(epoch+1, i+1, len(dataloaders['train'])))

EPOCH: 1, ITERATION: 1/782
EPOCH: 1, ITERATION: 11/782
EPOCH: 1, ITERATION: 21/782
EPOCH: 1, ITERATION: 31/782
EPOCH: 1, ITERATION: 41/782
EPOCH: 1, ITERATION: 51/782
EPOCH: 1, ITERATION: 61/782
EPOCH: 1, ITERATION: 71/782
EPOCH: 1, ITERATION: 81/782
EPOCH: 1, ITERATION: 91/782
EPOCH: 1, ITERATION: 101/782
EPOCH: 1, ITERATION: 111/782
EPOCH: 1, ITERATION: 121/782
EPOCH: 1, ITERATION: 131/782
EPOCH: 1, ITERATION: 141/782
EPOCH: 1, ITERATION: 151/782
EPOCH: 1, ITERATION: 161/782
EPOCH: 1, ITERATION: 171/782
EPOCH: 1, ITERATION: 181/782
EPOCH: 1, ITERATION: 191/782
EPOCH: 1, ITERATION: 201/782
EPOCH: 1, ITERATION: 211/782
EPOCH: 1, ITERATION: 221/782
EPOCH: 1, ITERATION: 231/782
EPOCH: 1, ITERATION: 241/782
EPOCH: 1, ITERATION: 251/782
EPOCH: 1, ITERATION: 261/782
EPOCH: 1, ITERATION: 271/782
EPOCH: 1, ITERATION: 281/782
EPOCH: 1, ITERATION: 291/782
EPOCH: 1, ITERATION: 301/782
EPOCH: 1, ITERATION: 311/782
EPOCH: 1, ITERATION: 321/782
EPOCH: 1, ITERATION: 331/782
EPOCH: 1, ITERATION: 341/

In [44]:
def validation(model, dataloaders):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    correct = 0
    total = 0
    with torch.no_grad():
        for i,(inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            if i % 10 == 0:
                print('\rIteration: {}/{}, Avg Acc: {:.4f}'.format(i+1, len(dataloaders['val']), (100 * correct / total)))
    print("Total Sample numbers: {}, Overall Acc: {:.2f}".format(total, (100 * correct / total)))

In [45]:
validation(model, dataloaders)

Iteration: 1/79, Avg Acc: 7.0312
Iteration: 11/79, Avg Acc: 13.2812
Iteration: 21/79, Avg Acc: 16.2202
Iteration: 31/79, Avg Acc: 15.2470
Iteration: 41/79, Avg Acc: 15.1296
Iteration: 51/79, Avg Acc: 18.8725
Iteration: 61/79, Avg Acc: 18.1609
Iteration: 71/79, Avg Acc: 17.4846
Total Sample numbers: 10000, Overall Acc: 17.51


In [46]:
attack = FastGradientMethod(estimator=classifier, eps=0.2)

In [57]:
def adv_validation(model, dataloaders):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    correct = 0
    total = 0
    for i,(inputs, labels) in enumerate(dataloaders['val']):
        inputs = inputs.numpy()
        labels = labels.numpy()

        # Add attack here
        inputs_adv = attack.generate(x=inputs)
        outputs = classifier.predict(inputs_adv)
        preds = np.argmax(outputs, axis=1)
        total += len(labels)
        correct += np.sum(preds == labels)
        if i % 10 == 0:
            print('\rIteration: {}/{}, Avg Acc for Adversarial inputs: {:.4f}'.format(i+1, len(dataloaders['val']), (100 * correct / total)))
    print("Total Sample numbers: {}, Overall Acc for Adversarial inputs: {:.2f}".format(total, (100 * correct / total)))

In [58]:
adv_validation(model, dataloaders)

Iteration: 1/79, Avg Acc for Adversarial inputs: 2.3438
Iteration: 11/79, Avg Acc for Adversarial inputs: 0.9233
Iteration: 21/79, Avg Acc for Adversarial inputs: 0.7812
Iteration: 31/79, Avg Acc for Adversarial inputs: 0.7560
Iteration: 41/79, Avg Acc for Adversarial inputs: 0.9718
Iteration: 51/79, Avg Acc for Adversarial inputs: 0.9651
Iteration: 61/79, Avg Acc for Adversarial inputs: 0.9221
Iteration: 71/79, Avg Acc for Adversarial inputs: 0.9023
Total Sample numbers: 10000, Overall Acc for Adversarial inputs: 0.91
